In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c obss-intern-competition-2025

 99% 990M/995M [00:09<00:00, 45.4MB/s]
100% 995M/995M [00:09<00:00, 111MB/s] 


In [4]:
import zipfile

with zipfile.ZipFile("obss-intern-competition-2025.zip", 'r') as zip_ref:
    zip_ref.extractall("obss_data")

In [5]:
import pandas as pd

train_df = pd.read_csv("obss_data/train.csv")
test_df = pd.read_csv("obss_data/test.csv")

print(train_df.sample(3))

       image_id                                            caption
8919       8919  This image is a music album cover featuring a ...
16055     16055  A large cardboard box with multiple sections i...
17059     17059  Poster titled "Split Loyalties: Fraser Valley ...


In [6]:
!pip install transformers accelerate timm bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00


In [7]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import torch

quant_config = BitsAndBytesConfig(load_in_8bit=True)

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

image = Image.open("obss_data/test/test/100000.jpg").convert("RGB")

inputs = processor(images=image, text="a photo of", return_tensors="pt").to("cuda", torch.float16)

generated_ids = model.generate(**inputs, max_new_tokens=50)
caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Caption:", caption)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Caption: a photo of a billboard advertising a restaurant



In [11]:
import os
from tqdm import tqdm
import pandas as pd
from PIL import Image

results = []

model.eval()

for image_id in tqdm(test_df['image_id']):
    image_path = os.path.join("obss_data/test/test", str(image_id) + ".jpg")
    try:
        raw_image = Image.open(image_path).convert('RGB')
        inputs = processor(images=raw_image, text="a photo of", return_tensors="pt").to("cuda")
        out = model.generate(**inputs, max_new_tokens=50)
        caption = processor.batch_decode(out, skip_special_tokens=True)[0]
        results.append((str(image_id) + ".jpg", caption))
    except Exception as e:
        print(f"({image_id}): {e}")
        results.append((str(image_id) + ".jpg", "error"))

submission_df = pd.DataFrame(results, columns=["image_id", "caption"])
submission_df.to_csv("submission.csv", index=False)

print("done.")

100%|██████████| 3771/3771 [1:09:43<00:00,  1.11s/it]

done.


In [12]:
submission_df.head(10)

,image_id,caption
0,100000.jpg,a photo of a billboard advertising a restaurant\n
1,100001.jpg,a photo of a person holding a knife on a beach\n
2,100002.jpg,a photo of a store in japan\n
3,100003.jpg,a photo of a red laptop on display at a store\n
4,100004.jpg,a photo of a cross stitch with the words blasp...
5,100005.jpg,a photo of a tall building with a sky backgrou...
6,100006.jpg,a photo of a store on the street\n
7,100007.jpg,a photo of a small airplane parked on the tarm...
8,100008.jpg,a photo of a shelf full of bottles of beer\n
9,100009.jpg,a photo of a book on a shelf in a library\n
